In [1]:
import os
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from skimage.feature import hog
import mahotas
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
 dataset_path = r'C:\Users\shash\Downloads\apple'
 img_size = (64, 64)

In [ ]:
X = []
y = []
labels = os.listdir(dataset_path)

In [ ]:
for label in labels:
    folder_path = os.path.join(dataset_path, label)
    if not os.path.isdir(folder_path):
        continue

    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue

        img = cv2.resize(img, img_size)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        features, _ = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), block_norm='L2-Hys',
                          visualize=True)

        moments = cv2.moments(gray)
        hu_moments = cv2.HuMoments(moments).flatten()

        haralick_features = mahotas.features.haralick(gray).mean(axis=0)

        hist_r = cv2.calcHist([img], [0], None, [256], [0, 256])
        hist_g = cv2.calcHist([img], [1], None, [256], [0, 256])
        hist_b = cv2.calcHist([img], [2], None, [256], [0, 256])
        color_hist = np.concatenate([hist_r.flatten(), hist_g.flatten(), hist_b.flatten()])

        combined_features = np.concatenate([features, hu_moments, haralick_features, color_hist])

        X.append(combined_features)
        y.append(label)

print(f"Total samples: {len(X)}") 


In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)  
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Combined Features + RF Accuracy: {accuracy * 100:.2f}%')
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(20, 18))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_,
            linewidths=0.5, linecolor='gray', annot_kws={"size": 20})

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix_rf.png', dpi=300, bbox_inches='tight')

plt.show()


In [ ]:

test_img_path = r'C:\Users\shash\OneDrive\Desktop\Plant Disease Detection\testingimages\AppleCedarRust2.JPG'

if not os.path.exists(test_img_path):
    print("File does not exist. Exiting.")
    exit()

img = cv2.imread(test_img_path)
if img is None:
    print("Failed to read image. Exiting.")
    exit()

img_size = (64, 64)
img_resized = cv2.resize(img, img_size)
gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

features, _ = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                  cells_per_block=(2, 2), block_norm='L2-Hys',
                  visualize=True)

moments = cv2.moments(gray)
hu_moments = cv2.HuMoments(moments).flatten()

haralick_features = mahotas.features.haralick(gray).mean(axis=0)

hist_r = cv2.calcHist([img_resized], [0], None, [256], [0, 256])
hist_g = cv2.calcHist([img_resized], [1], None, [256], [0, 256])
hist_b = cv2.calcHist([img_resized], [2], None, [256], [0, 256])
color_hist = np.concatenate([hist_r.flatten(), hist_g.flatten(), hist_b.flatten()])

combined_features = np.concatenate([features, hu_moments, haralick_features, color_hist])

combined_features_scaled = scaler.transform([combined_features])

prediction = rf.predict(combined_features_scaled)
predicted_label = prediction[0]

plt.imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))
plt.title(f"Predicted: {predicted_label}", fontsize=16)
plt.axis('off')
plt.show()
